In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from datetime import date, time, datetime
import math
import string

In [2]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np

In [3]:
load_dotenv()
db = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_SERVER')
port = os.getenv('POSTGRES_PORT')

In [4]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [5]:
# Accommodate query
sql = """SELECT id_port, code, port_name
FROM stg_pelni.master_port
where flag_ppss like 'Y'"""

In [6]:
# Read Query
df_sql = sqlio.read_sql_query(sql, conn)
d1 = sqlio.read_sql_query(sql, conn)

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_13528\304827915.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = sqlio.read_sql_query(sql, conn)
C:\Users\BN001602001\AppData\Local\Temp\ipykernel_13528\304827915.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  d1 = sqlio.read_sql_query(sql, conn)


In [7]:
# Read CSV
df = pd.read_csv('data_ship/total_ship/total_ship.csv')

In [8]:
df

,Unnamed: 0,dari,ke,bertolak,tiba,ship,temp_time,ruas,voyage
0,1,Kijang,Tg. Priok,2021-12-30 23:06:00,2022-01-01 10:50:00,UMSINI,0 days 10:27:00,1,1.0
1,2,Tg. Priok,Surabaya,2022-01-01 21:17:00,2022-01-03 02:34:00,UMSINI,0 days 04:29:00,2,1.0
2,3,Surabaya,Makassar,2022-01-03 07:03:00,2022-01-04 16:59:00,UMSINI,0 days 04:05:00,3,1.0
3,4,Makassar,Maumere,2022-01-04 21:04:00,2022-01-05 17:41:00,UMSINI,0 days 02:19:00,4,1.0
4,5,Maumere,Larantuka,2022-01-05 20:00:00,2022-01-06 02:06:00,UMSINI,0 days 01:08:00,5,1.0
...,...,...,...,...,...,...,...,...,...
9088,9089,Baubau,Raha,2022-12-29 06:20:00,2022-12-29 11:30:00,JETLINER,0 days 00:43:00,8,35.0
9089,9090,Raha,Kendari,2022-12-29 12:13:00,2022-12-29 20:06:00,JETLINER,0 days 13:14:00,9,35.0
9090,9091,Kendari,Raha,2022-12-30 09:20:00,2022-12-30 17:42:00,JETLINER,-1 days +23:48:00,10,35.0
9091,9092,Raha,Baubau,2022-12-30 17:30:00,2022-12-30 22:12:00,JETLINER,0 days 01:03:00,11,35.0


In [9]:
# Delete Data
df = df.drop(257)
df = df.drop(1864)
df = df.drop(1886)
df = df.drop(1888)
df = df.drop(2676)
df = df.drop(2692)
df = df.drop(3544)
df = df.drop(4569)
df = df.drop(4930)
df = df.drop(5609)
df = df.drop(6439)
df = df.drop(8882)
df = df.drop(9090)

df = df.drop(2087)
df = df.drop(2103)
df = df.drop(2119)
df = df.drop(2135)
df = df.drop(2151)
df = df.drop(2195)
df = df.drop(2225)
df = df.drop(2241)
df = df.drop(2257)
df = df.drop(2274)

df = df.drop(2088)
df = df.drop(2104)
df = df.drop(2120)
df = df.drop(2136)
df = df.drop(2152)
df = df.drop(2196)
df = df.drop(2226)
df = df.drop(2242)
df = df.drop(2258)
df = df.drop(2273)

In [10]:
# Split data time
df = df.dropna(subset=["temp_time"])
df['temp_time'] = df['temp_time'].astype(str)

df['a'] = df['temp_time'].str.split(' ').str[0]
df['b'] = df['temp_time'].str.split(' ').str[2]

df['time_1'] = df['b'].str.split(':').str[0]
df['time_2'] = df['b'].str.split(':').str[1]
df['time_2'] = '0.'+ df['time_2']

# Change days to hours
df.loc[df['a'] == '1', 'a'] = '24'
df.loc[df['a'] == '2', 'a'] = '48'
df.loc[df['a'] == '3', 'a'] = '72'
df.loc[df['a'] == '4', 'a'] = '96'
df.loc[df['a'] == '5', 'a'] = '120'
df.loc[df['a'] == '6', 'a'] = '144'
df.loc[df['a'] == '7', 'a'] = '168'


df['time_1'] = df['time_1'].astype(float)
df['time_2'] = df['time_2'].astype(float)
df['a'] = df['a'].astype(float)

In [11]:
# Calculation column time_1 + time_2 (output = Hours)
comb_time = []
for index, row in df.iterrows():
    A = row['time_1']
    B = row['time_2']
    result = A + B
    comb_time.append(result)
     
df['time'] = comb_time   

In [12]:
# Calculation column hours + minutes (output = float)
temp = []
for index, row in df.iterrows():
    A = row['a']
    B = row['time']
    result = A + B
    temp.append(result)
     
df['berth_time'] = temp   

In [13]:
# Loc columns
df['voyage'] = df['voyage'].astype(int)

df = df.loc[:, ['ship', 'dari', 'ke', 'bertolak', 'tiba', 'ruas', 'voyage',
       'berth_time']]

In [14]:
# Change Columns from index
df.loc[df.index == 616, "ke"] = "SEMARANG"
df.loc[df.index == 616, "dari"] = "PONTIANAK"

df.loc[df.index == 1332, "ke"] = "AMBON"
df.loc[df.index == 1332, "dari"] = "TERNATE"

In [15]:
# Change Name for columns 'ke'
df.loc[df["ke"] == "BAU BAU", "ke"] = "Bau-Bau/Murhum"
df.loc[df["ke"] == "BAU-BAU", "ke"] = "Bau-Bau/Murhum"
df.loc[df["ke"] == " BAU-BAU", "ke"] = "Bau-Bau/Murhum"
df.loc[df["ke"] == "Baubau", "ke"] = "Bau-Bau/Murhum"

df.loc[df["ke"] == " BAU BAU", "ke"] = "Bau-Bau/Murhum"
df.loc[df["ke"] == " BAU-BAU", "ke"] = "Bau-Bau/Murhum"
df.loc[df["ke"] == " BAUBAU", "ke"] = "Bau-Bau/Murhum"

df.loc[df["ke"] == "Tg. Priok", "ke"] = "Tanjung Priok"
df.loc[df["ke"] == "Maumere", "ke"] = "Maumere /Laurentius Say"
df.loc[df["ke"] == "FAK-FAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == "FAK FAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == " FAK-FAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == "FAKFAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == " FAKFAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == " FAK FAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == " Fak-Fak", "ke"] = "Fak-fak"
df.loc[df["ke"] == " FAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == "FAK", "ke"] = "Fak-fak"
df.loc[df["ke"] == "FAK ", "ke"] = "Fak-fak"
df.loc[df["ke"] == "AGATS", "ke"] = "Agats"
df.loc[df["ke"] == " O.B AGATS", "ke"] = "Agats"
df.loc[df["ke"] == "O.B AGATS", "ke"] = "Agats"
df.loc[df["ke"] == " AGATS", "ke"] = "Agats"
df.loc[df["ke"] == " O/B AGATS", "ke"] = "Agats"
df.loc[df["ke"] == " M.AGATS", "ke"] = "Agats"
df.loc[df["ke"] == "Tidore", "ke"] = "TIDORE"

df.loc[df["ke"] == " BALIKPAPAM", "ke"] = "Semayang"
df.loc[df["ke"] == "BALIKPAPAN", "ke"] = "Semayang"
df.loc[df["ke"] == "Balikpapan", "ke"] = "Semayang"
df.loc[df["ke"] == " BALIKPAPAN", "ke"] = "Semayang"

df.loc[df["ke"] == " O.B MERAUKE", "ke"] = "Merauke"
df.loc[df["ke"] == "O/B AGATS", "ke"] = "Agats"
df.loc[df["ke"] == " O.B MERAUKE", "ke"] = "Merauke"

df.loc[df["ke"] == " O.B. Tmika", "ke"] = "Timika"
df.loc[df["ke"] == "O.B.Kaimana", "ke"] = "Kaimana"
df.loc[df["ke"] == "O.B.MERAUKE", "ke"] = "Merauke"
df.loc[df["ke"] == " OB. MERAUKE", "ke"] = "Merauke"

df.loc[df["ke"] == "O.b Makasar", "ke"] = "Makassar"
df.loc[df["ke"] == "MAKASSAR", "ke"] = "Makassar"
df.loc[df["ke"] == " MAKASSAR", "ke"] = "Makassar"

df.loc[df["ke"] == "Kd. Kaimana", "ke"] = "Kaimana"
df.loc[df["ke"] == "Kd. Timika", "ke"] = "Timika"
df.loc[df["ke"] == "O.B. Tmika", "ke"] = "Timika"
df.loc[df["ke"] == "TIMIKA", "ke"] = "Timika"
df.loc[df["ke"] == " TIMIKA", "ke"] = "Timika"

df.loc[df["ke"] == "OB. MERAUKE", "ke"] = "Merauke"
df.loc[df["ke"] == "Tg.Perak", "ke"] = "Tanjung Perak"

df.loc[df["ke"] == "SURABAYA", "ke"] = "Tanjung Perak"
df.loc[df["ke"] == " SURABAYA LBH", "ke"] = "Tanjung Perak"
df.loc[df["ke"] == " SURABAYA", "ke"] = "Tanjung Perak"
df.loc[df["ke"] == "Surabaya", "ke"] = "Tanjung Perak"
df.loc[df["ke"] == "Surabayya", "ke"] = "Tanjung Perak"

df.loc[df["ke"] == "REDE JAKARTA", "ke"] = "Tanjung Priok"
df.loc[df["ke"] == "TG.PRIOK", "ke"] = "Tanjung Priok"
df.loc[df["ke"] == "TG PRIOK", "ke"] = "Tanjung Priok"

df.loc[df["ke"] == "SEMARANG", "ke"] = "Tanjung Emas"
df.loc[df["ke"] == " SEMARANG", "ke"] = "Tanjung Emas"
df.loc[df["ke"] == "Semarang", "ke"] = "Tanjung Emas"


In [16]:
df.loc[df["ke"] == "Tg. Balai K", "ke"] = "Tanjung Balai Karimun"
df.loc[df["ke"] == "Semarang", "ke"] = "Tanjung Emas"

df.loc[df["ke"] == "KARIMUN JAWA", "ke"] = "Legon Bajak, Karimun Jawa"
df.loc[df["ke"] == "KARIMUNJAWA", "ke"] = "Legon Bajak, Karimun Jawa"
df.loc[df["ke"] == "K.Jawa", "ke"] = "Legon Bajak, Karimun Jawa"

df.loc[df["ke"] == "Tg. Balai K", "ke"] = "Tanjung Balai Karimun"
df.loc[df["ke"] == " TG BALAI K", "ke"] = "Tanjung Balai Karimun"
df.loc[df["ke"] == " TG. BALAI K", "ke"] = "Tanjung Balai Karimun"
df.loc[df["ke"] == " AMBON", "ke"] = "Ambon"
df.loc[df["ke"] == "AMBON", "ke"] = "Ambon"

df.loc[df["ke"] == " MERAUKE", "ke"] = "Merauke"
df.loc[df["ke"] == "MERAUKE", "ke"] = "Merauke"
df.loc[df["ke"] == " KAIMANA", "ke"] = "Kaimana"
df.loc[df["ke"] == "KAIMANA", "ke"] = "Kaimana"

df.loc[df["ke"] == " BANGGAI", "ke"] = "Luwuk"
df.loc[df["ke"] == " BATAM", "ke"] = "Batam/Batu Ampar"
df.loc[df["ke"] == " BATULICIN", "ke"] = "Samudera, Batulicin"
df.loc[df["ke"] == " BATULICIN", "ke"] = "Samudera, Batulicin"
df.loc[df["ke"] == " BELAWAN", "ke"] = "Belawan"
df.loc[df["ke"] == " BENOA", "ke"] = "Benoa"
df.loc[df["ke"] == " BIAK", "ke"] = "Biak"
df.loc[df["ke"] == " BIMA", "ke"] = "Bima"
df.loc[df["ke"] == " BITUNG", "ke"] = "Bitung"
df.loc[df["ke"] == " DOBO", "ke"] = "Dobo"
df.loc[df["ke"] == " ENDE", "ke"] = "Ende"
df.loc[df["ke"] == " GESER", "ke"] = "Geser"
df.loc[df["ke"] == " KIJANG", "ke"] = "Sei Kolak Kijang"
df.loc[df["ke"] == " JAYAPURA", "ke"] = "Jayapura"
df.loc[df["ke"] == " KUMAI", "ke"] = "Kumai"
df.loc[df["ke"] == " LABUAN BAJO", "ke"] = "Labuan Bajo"
df.loc[df["ke"] == " LABUHAN BAJO", "ke"] = "Labuan Bajo"
df.loc[df["ke"] == " LARANTUKA", "ke"] = "Larantuka"
df.loc[df["ke"] == " LEWOLEBA", "ke"] = "Lewoleba"
df.loc[df["ke"] == " MANOKWARI", "ke"] = "Manokwari"
df.loc[df["ke"] == " MAUMERE", "ke"] = "Maumere /Laurentius Say"
df.loc[df["ke"] == " NABIRE", "ke"] = "Nabire"
df.loc[df["ke"] == " NAMLEA", "ke"] = "Namlea"
df.loc[df["ke"] == " KUPANG", "ke"] = "Kupang"
df.loc[df["ke"] == " NAMROLE", "ke"] = "Namrole"
df.loc[df["ke"] == " PANJANG", "ke"] = "Panjang"
df.loc[df["ke"] == " PANTOLOAN", "ke"] = "Pantoloan"
df.loc[df["ke"] == " PARE PARE", "ke"] = "Pare-Pare"
df.loc[df["ke"] == " RE.BITUNG", "ke"] = "Bitung"
df.loc[df["ke"] == " REDE BITUNG", "ke"] = "Bitung"


In [17]:
df.loc[df["ke"] == " NUNUKAN", "ke"] = "Nunukan/Tunon Taka"
df.loc[df["ke"] == " ROTE", "ke"] = "Baa, Rote"
df.loc[df["ke"] == " SABU", "ke"] = "Seba, Sabu"
df.loc[df["ke"] == " SANANA", "ke"] = "Sanana"
df.loc[df["ke"] == " SERUI", "ke"] = "Serui"
df.loc[df["ke"] == " SORONG", "ke"] = "Sorong"
df.loc[df["ke"] == " TARAKAN", "ke"] = "Tarakan"
df.loc[df["ke"] == " TERNATE", "ke"] = "Ternate"
df.loc[df["ke"] == " TUAL", "ke"] = "Tual"
df.loc[df["ke"] == " WAIKELO", "ke"] = "Waikelo"

df.loc[df["ke"] == " KR.JAMUANG", "ke"] = "KR.JAMUANG"
df.loc[df["ke"] == "GN SITOLI", "ke"] = "Gunung Sitoli"
df.loc[df["ke"] == " WANCI", "ke"] = "Wanci"
df.loc[df["ke"] == " WAINGAPU", "ke"] = "Waingapu"
df.loc[df["ke"] == " WASIOR", "ke"] = "Wasior"
df.loc[df["ke"] == "BATAM", "ke"] = "Batam/Batu Ampar"
df.loc[df["ke"] == "BATULICIN", "ke"] = "Samudera, Batulicin"
df.loc[df["ke"] == "BATULUCIN", "ke"] = "Samudera, Batulicin"
df.loc[df["ke"] == "BAU-BAU ", "ke"] = "Bau-Bau/Murhum"
df.loc[df["ke"] == "BELAWAN", "ke"] = "Belawan"
df.loc[df["ke"] == "BIAK", "ke"] = "Biak"
df.loc[df["ke"] == "BIMA", "ke"] = "Bima"
df.loc[df["ke"] == "BITUNG", "ke"] = "Bitung"
df.loc[df["ke"] == "BONTANG", "ke"] = "Lhok Tuan"
df.loc[df["ke"] == "Batam ", "ke"] = "Batam/Batu Ampar"
df.loc[df["ke"] == "Batam", "ke"] = "Batam/Batu Ampar"
df.loc[df["ke"] == "Bau-Bau", "ke"] = "Bau-Bau/Murhum"
df.loc[df["ke"] == "Blinyu", "ke"] = "Belinyu"

df.loc[df["ke"] == "DOBO", "ke"] = "Dobo"
df.loc[df["ke"] == "ENDE", "ke"] = "Ende"
df.loc[df["ke"] == "Fak-Fak", "ke"] = "Fak-fak"
df.loc[df["ke"] == "GESER", "ke"] = "Geser"
df.loc[df["ke"] == "KIJANG", "ke"] = "Sei Kolak Kijang"
df.loc[df["ke"] == "Kijang", "ke"] = "Sei Kolak Kijang"
df.loc[df["ke"] == "JAYAPURA", "ke"] = "Jayapura"
df.loc[df["ke"] == "LABUAN BAJO", "ke"] = "Labuan Bajo"
df.loc[df["ke"] == "L. Bajo", "ke"] = "Labuan Bajo"
df.loc[df["ke"] == "LARANTUKA", "ke"] = "Larantuka"
df.loc[df["ke"] == "LEMBAR", "ke"] = "Lembar"
df.loc[df["ke"] == "MANOKWARI", "ke"] = "Manokwari"
df.loc[df["ke"] == "MAUMERE", "ke"] = "Maumere /Laurentius Say"
df.loc[df["ke"] == "MOA", "ke"] = "Moa"
df.loc[df["ke"] == "NABIRE", "ke"] = "Nabire"
df.loc[df["ke"] == "NAMLEA", "ke"] = "Namlea"
df.loc[df["ke"] == "NAMROLE", "ke"] = "Namrole"
df.loc[df["ke"] == "NUNUKAN", "ke"] = "Nunukan/Tunon Taka"
df.loc[df["ke"] == "Nunukan", "ke"] = "Nunukan/Tunon Taka"
df.loc[df["ke"] == " KALABAHI", "ke"] = "Kalabahi"

df.loc[df["ke"] == "TERNATE", "ke"] = "Ternate"
df.loc[df["ke"] == "P0NTIANAK", "ke"] = "Pontianak"
df.loc[df["ke"] == "PADANG", "ke"] = "Teluk Bayur"
df.loc[df["ke"] == "PANJANG", "ke"] = "Panjang"
df.loc[df["ke"] == "WASIOR", "ke"] = "Wasior"
df.loc[df["ke"] == "PANTOLOAN", "ke"] = "Pantoloan"
df.loc[df["ke"] == "PARE PARE", "ke"] = "Pare-Pare"
df.loc[df["ke"] == "PARE-PARE", "ke"] = "Pare-Pare"
df.loc[df["ke"] == "Pare's", "ke"] = "Pare-Pare"
df.loc[df["ke"] == "PONTIANAK", "ke"] = "Pontianak"
df.loc[df["ke"] == "WAINGAPU", "ke"] = "Waingapu"
df.loc[df["ke"] == "WAIKELO", "ke"] = "Waikelo"
df.loc[df["ke"] == "TUAL", "ke"] = "Tual"
df.loc[df["ke"] == "TG PANDAN", "ke"] = "Tanjung Pandan"
df.loc[df["ke"] == "TG. BALAI K", "ke"] = "Tanjung Balai Karimun"
df.loc[df["ke"] == "T u a l", "ke"] = "Tual"
df.loc[df["ke"] == "TARAKAN", "ke"] = "Tarakan"
df.loc[df["ke"] == "TEPA", "ke"] = "Tepa"
df.loc[df["ke"] == "SORONG", "ke"] = "Sorong"
df.loc[df["ke"] == "SERUI", "ke"] = "Serui"
df.loc[df["ke"] == "SIBOLGA", "ke"] = "Sibolga"
df.loc[df["ke"] == "SANANA", "ke"] = "Sanana"
df.loc[df["ke"] == "SAUMLAKI", "ke"] = "Saumlaki"
df.loc[df["ke"] == "SAMPIT", "ke"] = "Sampit"
df.loc[df["ke"] == "BANDA", "ke"] = "Banda Naira"
df.loc[df["ke"] == "Awerange", "ke"] = "Awerange/Barru"
df.loc[df["ke"] == " BACAN", "ke"] = "BACAN"

df.loc[df["ke"] == " KR. JAMUANG", "ke"] = "KR.JAMUANG"
df.loc[df["ke"] == " RD.JAMUANG", "ke"] = "KR.JAMUANG"
df.loc[df["ke"] == " KA.TIDORE", "ke"] = "TIDORE"
df.loc[df["ke"] == " RE.TIDORE", "ke"] = "TIDORE"
df.loc[df["ke"] == " TIDORE", "ke"] = "TIDORE"
df.loc[df["ke"] == "KISAR", "ke"] = "Kisar"
df.loc[df["ke"] == "KUMAI", "ke"] = "Kumai"
df.loc[df["ke"] == "KUPANG", "ke"] = "Kupang"

In [18]:
# Change Name for columns 'dari'
df.loc[df["dari"] == "BAU BAU", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "BAU-BAU", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "BAU-BAU ", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "Baubau", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "BAU BAU ", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "BAU-BAU ", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "BAUBAU ", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "Tg. Priok", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "Maumere", "dari"] = "Maumere /Laurentius Say"
df.loc[df["dari"] == "FAK-FAK", "dari"] = "Fak-fak"
df.loc[df["dari"] == "FAK FAK", "dari"] = "Fak-fak"
df.loc[df["dari"] == "FAK-FAK ", "dari"] = "Fak-fak"
df.loc[df["dari"] == "FAKFAK ", "dari"] = "Fak-fak"
df.loc[df["dari"] == "FAKFAK ", "dari"] = "Fak-fak"
df.loc[df["dari"] == "FAK FAK ", "dari"] = "Fak-fak"
df.loc[df["dari"] == "Fak-Fak ", "dari"] = "Fak-fak"
df.loc[df["dari"] == "FAK ", "dari"] = "Fak-fak"
df.loc[df["dari"] == "FAK", "dari"] = "Fak-fak"
df.loc[df["dari"] == "AGATS", "dari"] = "Agats"
df.loc[df["dari"] == "O.B AGATS ", "dari"] = "Agats"
df.loc[df["dari"] == "O.B AGATS", "dari"] = "Agats"
df.loc[df["dari"] == "AGATS ", "dari"] = "Agats"
df.loc[df["dari"] == "O/B AGATS ", "dari"] = "Agats"
df.loc[df["dari"] == "M.AGATS ", "dari"] = "Agats"
df.loc[df["dari"] == "BALIKPAPAM ", "dari"] = "Semayang"
df.loc[df["dari"] == "BALIKPAPAN", "dari"] = "Semayang"
df.loc[df["dari"] == "Balikpapan", "dari"] = "Semayang"
df.loc[df["dari"] == "BALIKPAPAN ", "dari"] = "Semayang"
df.loc[df["dari"] == "O.B MERAUKE ", "dari"] = "Merauke"
df.loc[df["dari"] == "O/B AGATS", "dari"] = "Agats"
df.loc[df["dari"] == "O.B MERAUKE ", "dari"] = "Merauke"
df.loc[df["dari"] == "O.B. Tmika ", "dari"] = "Timika"
df.loc[df["dari"] == "O.B.Kaimana", "dari"] = "Kaimana"
df.loc[df["dari"] == "O.B.MERAUKE", "dari"] = "Merauke"
df.loc[df["dari"] == "OB. MERAUKE ", "dari"] = "Merauke"
df.loc[df["dari"] == "O.b Makasar", "dari"] = "Makassar"
df.loc[df["dari"] == "MAKASSAR", "dari"] = "Makassar"
df.loc[df["dari"] == "MAKASSAR ", "dari"] = "Makassar"
df.loc[df["dari"] == "Kd. Kaimana", "dari"] = "Kaimana"
df.loc[df["dari"] == "Kd. Timika", "dari"] = "Timika"
df.loc[df["dari"] == "O.B. Tmika", "dari"] = "Timika"
df.loc[df["dari"] == "TIMIKA", "dari"] = "Timika"
df.loc[df["dari"] == "TIMIKA ", "dari"] = "Timika"
df.loc[df["dari"] == "OB. MERAUKE", "dari"] = "Merauke"
df.loc[df["dari"] == "Tg.Perak", "dari"] = "Tanjung Perak"
df.loc[df["dari"] == "SURABAYA", "dari"] = "Tanjung Perak"
df.loc[df["dari"] == "SURABAYA LBH ", "dari"] = "Tanjung Perak"
df.loc[df["dari"] == "SURABAYA ", "dari"] = "Tanjung Perak"
df.loc[df["dari"] == "Surabaya", "dari"] = "Tanjung Perak"
df.loc[df["dari"] == "Surabayya", "dari"] = "Tanjung Perak"
df.loc[df["dari"] == "REDE JAKARTA", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "TG.PRIOK", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "TG PRIOK", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "SEMARANG", "dari"] = "Tanjung Emas"
df.loc[df["dari"] == "SEMARANG ", "dari"] = "Tanjung Emas"
df.loc[df["dari"] == "Semarang", "dari"] = "Tanjung Emas"


In [19]:
df.loc[df["dari"] == "Tg. Balai K", "dari"] = "Tanjung Balai Karimun"
df.loc[df["dari"] == "Semarang", "dari"] = "Tanjung Emas"

df.loc[df["dari"] == "KARIMUN JAWA", "dari"] = "Legon Bajak, Karimun Jawa"
df.loc[df["dari"] == "KARIMUNJAWA", "dari"] = "Legon Bajak, Karimun Jawa"
df.loc[df["dari"] == "K.Jawa", "dari"] = "Legon Bajak, Karimun Jawa"

df.loc[df["dari"] == "Tg. Balai K", "dari"] = "Tanjung Balai Karimun"
df.loc[df["dari"] == "TG BALAI K ", "dari"] = "Tanjung Balai Karimun"
df.loc[df["dari"] == "TG. BALAI K ", "dari"] = "Tanjung Balai Karimun"
df.loc[df["dari"] == "AMBON ", "dari"] = "Ambon"
df.loc[df["dari"] == "AMBON", "dari"] = "Ambon"

df.loc[df["dari"] == "MERAUKE ", "dari"] = "Merauke"
df.loc[df["dari"] == "MERAUKE", "dari"] = "Merauke"
df.loc[df["dari"] == "KAIMANA ", "dari"] = "Kaimana"
df.loc[df["dari"] == "KAIMANA", "dari"] = "Kaimana"

df.loc[df["dari"] == "BANGGAI ", "dari"] = "Luwuk"
df.loc[df["dari"] == "BATAM ", "dari"] = "Batam/Batu Ampar"
df.loc[df["dari"] == "BATULICIN ", "dari"] = "Samudera, Batulicin"
df.loc[df["dari"] == "BATULICIN ", "dari"] = "Samudera, Batulicin"
df.loc[df["dari"] == "BELAWAN ", "dari"] = "Belawan"
df.loc[df["dari"] == "BENOA ", "dari"] = "Benoa"
df.loc[df["dari"] == "BIAK ", "dari"] = "Biak"
df.loc[df["dari"] == "BIMA ", "dari"] = "Bima"
df.loc[df["dari"] == "BITUNG ", "dari"] = "Bitung"
df.loc[df["dari"] == "DOBO ", "dari"] = "Dobo"
df.loc[df["dari"] == "ENDE ", "dari"] = "Ende"
df.loc[df["dari"] == "GESER ", "dari"] = "Geser"
df.loc[df["dari"] == "KIJANG ", "dari"] = "Sei Kolak Kijang"
df.loc[df["dari"] == "JAYAPURA ", "dari"] = "Jayapura"
df.loc[df["dari"] == "KUMAI ", "dari"] = "Kumai"
df.loc[df["dari"] == "LABUAN BAJO ", "dari"] = "Labuan Bajo"
df.loc[df["dari"] == "LABUHAN BAJO ", "dari"] = "Labuan Bajo"
df.loc[df["dari"] == "LARANTUKA ", "dari"] = "Larantuka"
df.loc[df["dari"] == "LEWOLEBA ", "dari"] = "Lewoleba"
df.loc[df["dari"] == "MANOKWARI ", "dari"] = "Manokwari"
df.loc[df["dari"] == "MAUMERE ", "dari"] = "Maumere /Laurentius Say"
df.loc[df["dari"] == "NABIRE ", "dari"] = "Nabire"
df.loc[df["dari"] == "NAMLEA ", "dari"] = "Namlea"
df.loc[df["dari"] == "KUPANG ", "dari"] = "Kupang"
df.loc[df["dari"] == "NAMROLE ", "dari"] = "Namrole"
df.loc[df["dari"] == "PANJANG ", "dari"] = "Panjang"
df.loc[df["dari"] == "PANTOLOAN ", "dari"] = "Pantoloan"
df.loc[df["dari"] == "PARE PARE ", "dari"] = "Pare-Pare"
df.loc[df["dari"] == "RE.BITUNG ", "dari"] = "Bitung"
df.loc[df["dari"] == "REDE BITUNG ", "dari"] = "Bitung"


In [20]:
df.loc[df["dari"] == "NUNUKAN ", "dari"] = "Nunukan/Tunon Taka"
df.loc[df["dari"] == "ROTE ", "dari"] = "Baa, Rote"
df.loc[df["dari"] == "SABU ", "dari"] = "Seba, Sabu"
df.loc[df["dari"] == "SANANA ", "dari"] = "Sanana"
df.loc[df["dari"] == "SERUI ", "dari"] = "Serui"
df.loc[df["dari"] == "SORONG ", "dari"] = "Sorong"
df.loc[df["dari"] == "TARAKAN ", "dari"] = "Tarakan"
df.loc[df["dari"] == "TERNATE ", "dari"] = "Ternate"
df.loc[df["dari"] == "TUAL ", "dari"] = "Tual"
df.loc[df["dari"] == "WAIKELO ", "dari"] = "Waikelo"

df.loc[df["dari"] == "KR.JAMUANG ", "dari"] = "KR.JAMUANG"
df.loc[df["dari"] == "GN SITOLI", "dari"] = "Gunung Sitoli"
df.loc[df["dari"] == "WANCI ", "dari"] = "Wanci"
df.loc[df["dari"] == "WAINGAPU ", "dari"] = "Waingapu"
df.loc[df["dari"] == "WASIOR ", "dari"] = "Wasior"
df.loc[df["dari"] == "BATAM", "dari"] = "Batam/Batu Ampar"
df.loc[df["dari"] == "BATULICIN", "dari"] = "Samudera, Batulicin"
df.loc[df["dari"] == "BATULUCIN", "dari"] = "Samudera, Batulicin"
df.loc[df["dari"] == "BAU-BAU ", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "BELAWAN", "dari"] = "Belawan"
df.loc[df["dari"] == "BIAK", "dari"] = "Biak"
df.loc[df["dari"] == "BIMA", "dari"] = "Bima"
df.loc[df["dari"] == "BITUNG", "dari"] = "Bitung"
df.loc[df["dari"] == "BONTANG", "dari"] = "Lhok Tuan"
df.loc[df["dari"] == "Batam ", "dari"] = "Batam/Batu Ampar"
df.loc[df["dari"] == "Batam", "dari"] = "Batam/Batu Ampar"
df.loc[df["dari"] == "Bau-Bau", "dari"] = "Bau-Bau/Murhum"
df.loc[df["dari"] == "Blinyu", "dari"] = "Belinyu"

df.loc[df["dari"] == "DOBO", "dari"] = "Dobo"
df.loc[df["dari"] == "ENDE", "dari"] = "Ende"
df.loc[df["dari"] == "Fak-Fak", "dari"] = "Fak-fak"
df.loc[df["dari"] == "GESER", "dari"] = "Geser"
df.loc[df["dari"] == "KIJANG", "dari"] = "Sei Kolak Kijang"
df.loc[df["dari"] == "Kijang", "dari"] = "Sei Kolak Kijang"
df.loc[df["dari"] == "JAYAPURA", "dari"] = "Jayapura"
df.loc[df["dari"] == "LABUAN BAJO", "dari"] = "Labuan Bajo"
df.loc[df["dari"] == "L. Bajo", "dari"] = "Labuan Bajo"
df.loc[df["dari"] == "LARANTUKA", "dari"] = "Larantuka"
df.loc[df["dari"] == "LEMBAR", "dari"] = "Lembar"
df.loc[df["dari"] == "MANOKWARI", "dari"] = "Manokwari"
df.loc[df["dari"] == "MAUMERE", "dari"] = "Maumere /Laurentius Say"
df.loc[df["dari"] == "MOA", "dari"] = "Moa"
df.loc[df["dari"] == "NABIRE", "dari"] = "Nabire"
df.loc[df["dari"] == "NAMLEA", "dari"] = "Namlea"
df.loc[df["dari"] == "NAMROLE", "dari"] = "Namrole"
df.loc[df["dari"] == "NUNUKAN", "dari"] = "Nunukan/Tunon Taka"
df.loc[df["dari"] == "Nunukan", "dari"] = "Nunukan/Tunon Taka"
df.loc[df["dari"] == "KALABAHI ", "dari"] = "Kalabahi"

df.loc[df["dari"] == "TERNATE", "dari"] = "Ternate"
df.loc[df["dari"] == "P0NTIANAK", "dari"] = "Pontianak"
df.loc[df["dari"] == "PADANG", "dari"] = "Teluk Bayur"
df.loc[df["dari"] == "PANJANG", "dari"] = "Panjang"
df.loc[df["dari"] == "WASIOR", "dari"] = "Wasior"
df.loc[df["dari"] == "PANTOLOAN", "dari"] = "Pantoloan"
df.loc[df["dari"] == "PARE PARE", "dari"] = "Pare-Pare"
df.loc[df["dari"] == "PARE-PARE", "dari"] = "Pare-Pare"
df.loc[df["dari"] == "Pare's", "dari"] = "Pare-Pare"
df.loc[df["dari"] == "PONTIANAK", "dari"] = "Pontianak"
df.loc[df["dari"] == "WAINGAPU", "dari"] = "Waingapu"
df.loc[df["dari"] == "WAIKELO", "dari"] = "Waikelo"
df.loc[df["dari"] == "TUAL", "dari"] = "Tual"
df.loc[df["dari"] == "TG PANDAN", "dari"] = "Tanjung Pandan"
df.loc[df["dari"] == "TG. BALAI K", "dari"] = "Tanjung Balai Karimun"
df.loc[df["dari"] == "T u a l", "dari"] = "Tual"
df.loc[df["dari"] == "TARAKAN", "dari"] = "Tarakan"
df.loc[df["dari"] == "TEPA", "dari"] = "Tepa"
df.loc[df["dari"] == "SORONG", "dari"] = "Sorong"
df.loc[df["dari"] == "SERUI", "dari"] = "Serui"
df.loc[df["dari"] == "SIBOLGA", "dari"] = "Sibolga"
df.loc[df["dari"] == "SANANA", "dari"] = "Sanana"
df.loc[df["dari"] == "SAUMLAKI", "dari"] = "Saumlaki"
df.loc[df["dari"] == "SAMPIT", "dari"] = "Sampit"
df.loc[df["dari"] == "BANDA", "dari"] = "Banda Naira"
df.loc[df["dari"] == "Awerange", "dari"] = "Awerange/Barru"
df.loc[df["dari"] == "BACAN ", "dari"] = "BACAN"

df.loc[df["dari"] == "KR. JAMUANG ", "dari"] = "KR.JAMUANG"
df.loc[df["dari"] == "RD.JAMUANG ", "dari"] = "KR.JAMUANG"
df.loc[df["dari"] == "KA.TIDORE ", "dari"] = "TIDORE"
df.loc[df["dari"] == "RE.TIDORE ", "dari"] = "TIDORE"
df.loc[df["dari"] == "TIDORE ", "dari"] = "TIDORE"
df.loc[df["dari"] == "KISAR", "dari"] = "Kisar"
df.loc[df["dari"] == "KUMAI", "dari"] = "Kumai"
df.loc[df["dari"] == "KUPANG", "dari"] = "Kupang"
df.loc[df["dari"] == "BENOA", "dari"] = "Benoa"
df.loc[df["dari"] == "BONTANG ", "dari"] = "Bontang"
df.loc[df["dari"] == "CIWANDAN ", "dari"] = "CIWANDAN"
df.loc[df["dari"] == "JAKARTA", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "JAKARTA ", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "KAIAMANA", "dari"] = "Kaimana"
df.loc[df["dari"] == "KALABAHI", "dari"] = "Kalabahi"
df.loc[df["dari"] == "LEMBAR ", "dari"] = "Lembar"
df.loc[df["dari"] == "MAKASAR", "dari"] = "Makassar"
df.loc[df["dari"] == "MAKASAR ", "dari"] = "Makassar"
df.loc[df["dari"] == "Manokwari ", "dari"] = "Manokwari"
df.loc[df["dari"] == "M.AGATS", "dari"] = "Agats"
df.loc[df["dari"] == "TG PRIOK ", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "Tg Priok", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "Tg.Priok", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "Priok", "dari"] = "Tanjung Priok"
df.loc[df["dari"] == "Tg.perak", "dari"] = "Tanjung Perak"
df.loc[df["dari"] == "\AMBON ", "dari"] = "Ambon"
df.loc[df["dari"] == "TIDORE", "dari"] = "Tidore"
df.loc[df["dari"] == "REDE BITUNG", "dari"] = "Bitung"
df.loc[df["dari"] == "PARE-PARE ", "dari"] = "Pare-Pare"
df.loc[df["dari"] == "O.B MERAUKE", "dari"] = "Merauke"
df.loc[df["dari"] == "PONTIANAK", "dari"] = "Pontianak"
df.loc[df["dari"] == "kalabahi", "dari"] = "Kalabahi"

In [21]:
df

,ship,dari,ke,bertolak,tiba,ruas,voyage,berth_time
0,UMSINI,Sei Kolak Kijang,Tanjung Priok,2021-12-30 23:06:00,2022-01-01 10:50:00,1,1,10.27
1,UMSINI,Tanjung Priok,Tanjung Perak,2022-01-01 21:17:00,2022-01-03 02:34:00,2,1,4.29
2,UMSINI,Tanjung Perak,Makassar,2022-01-03 07:03:00,2022-01-04 16:59:00,3,1,4.05
3,UMSINI,Makassar,Maumere /Laurentius Say,2022-01-04 21:04:00,2022-01-05 17:41:00,4,1,2.19
4,UMSINI,Maumere /Laurentius Say,Larantuka,2022-01-05 20:00:00,2022-01-06 02:06:00,5,1,1.08
...,...,...,...,...,...,...,...,...
9086,JETLINER,Bau-Bau/Murhum,Wanci,2022-12-27 22:50:00,2022-12-28 07:28:00,6,35,3.42
9087,JETLINER,Wanci,Bau-Bau/Murhum,2022-12-28 11:10:00,2022-12-28 21:15:00,7,35,9.05
9088,JETLINER,Bau-Bau/Murhum,Raha,2022-12-29 06:20:00,2022-12-29 11:30:00,8,35,0.43
9089,JETLINER,Raha,Kendari,2022-12-29 12:13:00,2022-12-29 20:06:00,9,35,13.14


In [22]:
# Group By column 'ke' for get average, min, and max
cleandf = df.groupby(['ke'])[["berth_time"]].aggregate([min, max,'mean'])

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_13528\1436116015.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'min' instead.
  cleandf = df.groupby(['ke'])[["berth_time"]].aggregate([min, max,'mean'])
C:\Users\BN001602001\AppData\Local\Temp\ipykernel_13528\1436116015.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'max' instead.
  cleandf = df.groupby(['ke'])[["berth_time"]].aggregate([min, max,'mean'])


In [23]:
cleandf.to_csv('data/mean.csv')

In [24]:
# Inner Join
# Change columns name
c1 = df_sql.rename(columns={'port_name': 'ke'})

# Mapping for code_des 
df['ke'] = df['ke'].astype(str)
c1['ke'] = c1['ke'].astype(str)

inner_join1 = pd.merge(df, 
                      c1, 
                      on ='ke', 
                      how ='inner')

# Change columns name
inner_join1 = inner_join1.rename(columns={'id_port': 'id_des'})
inner_join1 = inner_join1.rename(columns={'code': 'des_code'})
d1 = d1.rename(columns={'port_name': 'dari'})
d1['dari'] = d1['dari'].astype(str)

merge_data1 = pd.merge(inner_join1, 
                      d1, 
                      on ='dari',
                      how ='inner')
cleandf = merge_data1.rename(columns={'id_port':'id_org'})
cleandf = cleandf.rename(columns={'code': 'org_code'})
cleandf

,ship,dari,ke,bertolak,tiba,ruas,voyage,berth_time,id_des,des_code,id_org,org_code
0,UMSINI,Sei Kolak Kijang,Tanjung Priok,2021-12-30 23:06:00,2022-01-01 10:50:00,1,1,10.27,b2674a83-0456-4666-893d-7c7522f9232b,431,1334ec95-cc8a-4464-a263-0b0beee559da,251
1,UMSINI,Sei Kolak Kijang,Tanjung Priok,2022-01-13 18:49:00,2022-01-15 06:01:00,1,2,10.19,b2674a83-0456-4666-893d-7c7522f9232b,431,1334ec95-cc8a-4464-a263-0b0beee559da,251
2,UMSINI,Sei Kolak Kijang,Tanjung Priok,2022-01-28 00:11:00,2022-01-29 11:37:00,1,3,12.33,b2674a83-0456-4666-893d-7c7522f9232b,431,1334ec95-cc8a-4464-a263-0b0beee559da,251
3,UMSINI,Sei Kolak Kijang,Tanjung Priok,2022-02-11 02:00:00,2022-02-12 13:38:00,1,4,10.38,b2674a83-0456-4666-893d-7c7522f9232b,431,1334ec95-cc8a-4464-a263-0b0beee559da,251
4,UMSINI,Sei Kolak Kijang,Tanjung Priok,2022-02-25 05:53:00,2022-02-26 16:54:00,1,5,7.49,b2674a83-0456-4666-893d-7c7522f9232b,431,1334ec95-cc8a-4464-a263-0b0beee559da,251
...,...,...,...,...,...,...,...,...,...,...,...,...
7513,WILIS,"Seba, Sabu","Baa, Rote",2022-07-27 08:00:00,2022-07-27 19:45:00,8,15,2.15,395f3f65-4df6-4716-af13-2d5f0e02bdd7,696,5463607b-dcab-4475-be30-e3a91d051e95,697
7514,WILIS,"Seba, Sabu","Baa, Rote",2022-08-24 10:00:00,2022-08-24 20:15:00,8,17,1.45,395f3f65-4df6-4716-af13-2d5f0e02bdd7,696,5463607b-dcab-4475-be30-e3a91d051e95,697
7515,WILIS,"Seba, Sabu","Baa, Rote",2022-09-21 10:00:00,2022-09-21 19:40:00,8,19,1.50,395f3f65-4df6-4716-af13-2d5f0e02bdd7,696,5463607b-dcab-4475-be30-e3a91d051e95,697
7516,WILIS,"Seba, Sabu","Baa, Rote",2022-10-19 09:30:00,2022-10-19 19:00:00,8,21,2.30,395f3f65-4df6-4716-af13-2d5f0e02bdd7,696,5463607b-dcab-4475-be30-e3a91d051e95,697


In [25]:
cleandf.to_csv('data/final_LPK.csv')

In [26]:
# Change Typedata
cleandf['ke'] = cleandf['ke'].astype(str)
cleandf['dari'] = cleandf['dari'].astype(str)

cleandf = cleandf.loc[:, ['ke','berth_time']]
cleandf

,ke,berth_time
0,Tanjung Priok,10.27
1,Tanjung Priok,10.19
2,Tanjung Priok,12.33
3,Tanjung Priok,10.38
4,Tanjung Priok,7.49
...,...,...
7513,"Baa, Rote",2.15
7514,"Baa, Rote",1.45
7515,"Baa, Rote",1.50
7516,"Baa, Rote",2.30


In [27]:
# Group By column 'ke' for get average, min, and max
cleandf = cleandf.groupby(['ke'])[["berth_time"]].aggregate([min, max,'mean'])

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_13528\2364834084.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'min' instead.
  cleandf = cleandf.groupby(['ke'])[["berth_time"]].aggregate([min, max,'mean'])
C:\Users\BN001602001\AppData\Local\Temp\ipykernel_13528\2364834084.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'max' instead.
  cleandf = cleandf.groupby(['ke'])[["berth_time"]].aggregate([min, max,'mean'])


In [28]:
cleandf

berth_time                 
                      min    max      mean
ke                                        
Agats                1.02   8.33  2.940000
Ambon                1.29  16.54  4.246475
Awerange/Barru       0.37   2.20  1.185652
Baa, Rote            1.15   4.45  2.743182
Banda Naira          0.14  33.51  8.271912
...                   ...    ...       ...
Tual                 1.18   6.40  3.114286
Waikelo              0.35   3.17  1.240000
Waingapu             1.00  10.07  2.822151
Wanci                0.47  24.24  2.620189
Wasior               0.45   2.40  0.992174

[75 rows x 3 columns]

In [29]:
# cleandf.to_csv('data/mean.csv')

In [30]:
df.columns.values

array(['ship', 'dari', 'ke', 'bertolak', 'tiba', 'ruas', 'voyage',
       'berth_time'], dtype=object)